# 7. Relationships between variable

到目前为止，我们每次只关注一个变量。在本章中，我们将开始探讨变量之间的关系。如果了解一个变量能让你获得关于另一个变量的信息，那么这两个变量就是相关的。例如，身高和体重是相关的——个子较高的人往往更重。当然，这种关系并非完美：也有矮而重的人和高而轻的人。但如果你试图猜测某人的体重，知道他们的身高会比不知道时更准确。

本章将介绍几种可视化变量关系的方法，以及一种量化关系强度的方法——相关性。

In [ ]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats.py")

# 7.1 Scatter Plots 散点图

如果你遇到一个数学能力超群的人，你会认为他们的语言能力是高于还是低于平均水平？一方面，你可能会认为人们往往专精于某一领域，因此某人在一个领域表现出色，在另一个领域可能就相对较弱。另一方面，你可能会认为一个总体聪明的人在这两个领域都会高于平均水平。让我们来探究一下实际情况如何。

我们将使用 1997 年全国青年纵向调查（NLSY97）的数据，该调查“追踪了 8,984 名出生于 1980 年至 1984 年间的美国青年的生活”。公开数据集包含了参与者在多项标准化测试中的分数，包括大学入学最常用的 SAT 和 ACT 考试。由于考生在数学和语言部分分别获得分数，我们可以利用这些数据来探索数学能力和语言能力之间的关系。

我使用 NLS Investigator 创建了一个包含我将用于此分析的变量的数据摘录。在他们的许可下，我可以重新分发这个摘录。下载数据的说明在本章的笔记本中。

In [ ]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/nlsy97-extract.csv.gz")

我们可以使用 read_csv 来读取数据，并使用 replace 将缺失数据的特殊代码替换为 np.nan 。

In [ ]:
import pandas as pd
import numpy as np

missing_codes = [-1, -2, -3, -4, -5]
nlsy = pd.read_csv("nlsy97-extract.csv.gz").replace(missing_codes, np.nan)
nlsy.shape

In [ ]:
nlsy.head()

该 DataFrame 包含了调查中 8984 名参与者的每一行数据，以及我选择的 34 个变量中的每一列。列名本身意义不大，因此我们将用更具解释性的名称替换我们将使用的那些列。

In [ ]:
nlsy["sat_verbal"] = nlsy["R9793800"]
nlsy["sat_math"] = nlsy["R9793900"]

两列都包含一些小于 200 的值，这是不可能的，因为 200 是最低分数，因此我们将用 np.nan 替换它们。

In [ ]:
columns = ["sat_verbal", "sat_math"]

for column in columns:
    invalid = nlsy[column] < 200
    nlsy.loc[invalid, column] = np.nan

接下来，我们将使用 dropna 来筛选出两个分数都有效的行。

In [ ]:
nlsy_valid = nlsy.dropna(subset=columns).copy()
nlsy_valid.shape

SAT 分数经过标准化处理，其平均分为 500，标准差为 100。在 NLSY 样本中，这些均值和标准差与这些数值非常接近。

In [ ]:
sat_verbal = nlsy_valid["sat_verbal"]
sat_verbal.mean(), sat_verbal.std()

In [ ]:
sat_math = nlsy_valid["sat_math"]
sat_math.mean(), sat_math.std()

现在，为了观察这些变量之间是否存在关系，让我们来看一下散点图。

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(sat_verbal, sat_math)
plt.xlabel("SAT Verbal")
plt.ylabel("SAT Math")
plt.show()


使用 scatter 函数的默认选项，我们可以看出变量间关系的大致形态。在测试的某一部分表现良好的人，往往在其他部分也表现更佳。

然而，这个版本的图表存在过度绘制的问题，这意味着有大量重叠的点，这可能会对关系的理解产生误导。点密度最高的中心区域并没有应有的那么暗——相比之下，极端值却显得比应有的更暗。过度绘制往往会给予异常值过多的视觉权重。

In [ ]:
plt.scatter(sat_verbal, sat_math, s=5)
plt.xlabel("SAT Verbal")
plt.ylabel("SAT Math")
plt.show()


现在我们可以看到，标记点按行和列对齐，因为分数被四舍五入到最接近的 10 的倍数。在这个过程中，一些信息丢失了。



我们无法恢复那些丢失的信息，但可以通过抖动数据来最小化其对散点图的影响，即添加随机噪声以抵消四舍五入带来的影响。以下函数接收一个序列，并通过添加均值为 0、给定标准差的正态分布随机值来抖动数据，最终返回一个 NumPy 数组。

In [ ]:
def jitter(seq, std=1):
    n = len(seq)
    return np.random.normal(0, std, n) + seq

如果我们以 3 的标准差对分数进行抖动处理，散点图中的行和列将不再可见。

In [ ]:
sat_verbal_jittered = jitter(sat_verbal, 3)
sat_math_jittered = jitter(sat_math, 3)

In [ ]:
plt.scatter(sat_verbal_jittered, sat_math_jittered, s=5)
plt.xlabel("SAT Verbal")
plt.ylabel("SAT Math")
plt.show()

抖动减少了四舍五入的视觉影响，并使变量间的关系形态更加清晰。但通常，抖动应仅用于可视化目的，避免在分析中使用经过抖动的数据。

在这个例子中，即使调整了标记点大小并对数据进行了抖动处理，仍然存在一些重叠绘制的问题。因此，让我们再尝试一种方法：我们可以使用 alpha 关键字来使标记点部分透明。

In [ ]:
plt.scatter(sat_verbal_jittered, sat_math_jittered, s=5, alpha=0.2)
plt.xlabel("SAT Verbal")
plt.ylabel("SAT Math")
plt.show()

通过透明度设置，重叠的数据点会显得更暗，因此暗度与密度成正比。

尽管散点图是一种简单且广泛使用的可视化工具，但要正确绘制却可能颇具挑战。通常，需要通过反复试验来调整标记大小、透明度以及抖动程度，才能找到最能清晰展现变量间关系的最佳视觉呈现方式。

## 7.2 Decile Plots 十分位数图

散点图能提供变量间关系的总体印象，但还有其他可视化方法能更深入地揭示关系的本质。十分位数图便是其中之一。

要生成十分位数图，我们需要按语言分数对受访者进行排序，并将其分为 10 组，即十分位数。我们可以使用 qcut 方法来计算这些十分位数。

In [ ]:
deciles = pd.qcut(nlsy_valid["sat_verbal"], 10, labels=False) + 1
deciles

> _`pd.qcut` 函数的用途: 它将给定的数据从小到大排序, 然后进行数据分箱操作. 返回结果是一个 Series, 索引是原始数据的索引, 值是该数据所属的分箱序号. (请注意分箱序号从 0 开始)._

In [ ]:
pd.qcut(nlsy_valid["sat_verbal"], 10, labels=False)

> _为了使分箱序号更符合人类的理解, 通过 +1 的方式将分箱序号的起始值 0 调整到 1, 此时分箱序号是 1...10_

In [ ]:
pd.qcut(nlsy_valid["sat_verbal"], 10, labels=False) + 1


In [ ]:
deciles.value_counts().sort_index()

每个十分位数的受访者数量大致相等。

现在我们可以使用 groupby 方法，根据 decile 将 DataFrame 分成若干组。

In [ ]:
df_groupby = nlsy_valid.groupby(deciles)
df_groupby.get_group(1).head()

我们可以从中选择 sat_math 列。

In [ ]:
series_groupby = df_groupby["sat_math"]
series_groupby.get_group(1).head()

我们可以使用 quantile 函数来计算每组中的第 10、第 50 和第 90 百分位数。

In [ ]:
low = series_groupby.quantile(0.1)
median = series_groupby.quantile(0.5)
high = series_groupby.quantile(0.9)

low

十分位数图展示了每个十分位组的这些百分位数。在下图中，线条代表中位数，阴影区域则显示了第 10 百分位与第 90 百分位之间的范围。

In [ ]:
xs = median.index
plt.fill_between(xs, low, high, alpha=0.2)
plt.plot(xs, median, label="median")
plt.xlabel("SAT Verbal Decile")
plt.ylabel("SAT Math")
plt.legend()
plt.show()


_上面的十分位图有一些不容易理解或者容易理解错误的地方: 首先, 使用 **言语分数** 进行 `10分位` 分箱; 接着, 使用 **数学分数** 获取每组中第 10, 50, 90 的百分位数据, 其中: 第 10 和 90 百分位数据用于绘制阴影部分, 第 50 百分位数用于绘制线条表示中位数_

> - _分组依据（x 轴来源）： 在代码中，使用 pd.qcut(nlsy_valid["sat_verbal"], 10, ...) 来创建十分位组。这意味着他是按照 **言语分数（Verbal score）** 将受访者从小到大排队，然后平分成 10 个组。_
> - _观察指标（y 轴来源）： 接着，作者通过 df_groupby["sat_math"] 提取了每个“言语分数组”里的数学分数（Math scores）_
> - _图表含义： 这个图表展示的是：对于处在不同 **言语分数等级（x 轴：SAT Verbal Decile）的同学，他们的数学分数（y 轴：SAT Math）** 分布情况（中位数、10% 和 90% 分位数）是怎样的_

另一种方法是，我们可以计算每个组的言语分数中位数，并将这些值绘制在 x 轴上，而不是十分位数。

In [ ]:
xs = df_groupby["sat_verbal"].median()

plt.fill_between(xs, low, high, alpha=0.2)
plt.plot(xs, median, color="C0", label="median")
plt.xlabel("SAT Verbal Median")
plt.ylabel("SAT Math")
plt.legend()
plt.show()


这些变量之间的关系似乎是线性的——也就是说，中位数语文成绩每增加一次，中位数数学成绩也大致相应地增加。

更广泛地说，我们可以将受访者分成任意数量的组，不一定是 10 组，并且我们可以在每组中计算其他汇总统计量，而不仅仅是这些百分位数。